In [1]:
# -*- coding: utf-8 -*-
# """
# inference.ipynb
# Created on Oct Sept 30, 2024
# """

### Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# !pip install numpy==1.19.5 --quiet

### CUDA and Imports

In [4]:
# Check CUDA availability
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import os

torch.manual_seed(42)

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

print(f"Python Version | {sys.version}")
print(f"PyTorch Version | {torch.__version__}")

print("CUDA available |", torch.cuda.is_available())
print("CUDA device count |", torch.cuda.device_count())
print("Current device |", torch.cuda.current_device())


if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    print("Using CUDA device:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

# GPU INFORMATION
print("\nGPU INFORMATION")
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(f"{gpu_info}")

# MEMORY INFORMATION
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print("\nMEMORY INFORMATION")
print('Your runtime has {:.1f} GB of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


# from path import Path
# from os.path import exists

import os
import shutil
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial.distance
import math
import random
from pprint import pformat

import warnings
warnings.filterwarnings('ignore')

# sys.path.append(root_dir)

# Set the seed for Python's random module
random.seed(42)

# Set the seed for NumPy
np.random.seed(42)

Python Version | 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
PyTorch Version | 2.5.0+cu121
CUDA available | True
CUDA device count | 1
Current device | 0
Using CUDA device: Tesla T4

GPU INFORMATION
Wed Oct 30 14:51:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W 

### Setup Point2Roof

In [5]:
PROJ_DIR = "/content/drive/MyDrive/ColabNotebooks/IECO/point2roof"
REPO = os.path.join(PROJ_DIR, "POINT2ROOF")
print(f"REPO | {REPO}")

REPO | /content/drive/MyDrive/ColabNotebooks/IECO/point2roof/POINT2ROOF


In [6]:
# %cd /content
!git clone https://github.com/mm-mazhar/IECO_Point2Roof_Maz.git {REPO} --quiet
%cd {REPO}
print(f"\nCurrent Directory | {os.getcwd()}")

fatal: destination path '/content/drive/MyDrive/ColabNotebooks/IECO/point2roof/POINT2ROOF' already exists and is not an empty directory.
/content/drive/MyDrive/ColabNotebooks/IECO/point2roof/POINT2ROOF

Current Directory | /content/drive/MyDrive/ColabNotebooks/IECO/point2roof/POINT2ROOF


In [7]:
!pip install ninja --quiet
# !pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --quiet
# !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 8.3 MB/s eta 0:00:00


In [8]:
# Navigate to pc_util and rebuild
%cd pc_util

!python setup.py build_ext --inplace --quiet
!python setup.py install --quiet

%cd ..

####################################################
# %cd pc_util

# # Build the extension in-place
# !python setup.py build_ext --inplace

# # Install the package using pip (instead of setup.py install)
# !pip install .

# %cd ..

### Run Inference

In [ ]:

XYZ_FILE = "0dcbf0e7d298b9967fdca389c394133e18d7e4b1"
POINT_FILE: str = os.path.join(PROJ_DIR, "dataset_colob", "points", XYZ_FILE, "points.xyz")
MODEL_CONFIG: str = os.path.join(REPO, "model_cfg.yaml")                      
INFERENCE_DIR: str = os.path.join(REPO, "inference_output")
CHECKPOINT: str = "./checkpoint_epoch_90.pth"


print(f"POINT_FILE | {POINT_FILE}")
print(f"MODEL_CONFIG | {MODEL_CONFIG}")
print(f"INFERENCE_DIR | {INFERENCE_DIR}")
print(f"CHECKPOINT | {CHECKPOINT}")

In [11]:
!python ./inference.py --file_path {POINT_FILE} --cfg_file {MODEL_CONFIG} --batch_size 1 --gpu 0 --output_dir {INFERENCE_DIR} --ckpt_file {CHECKPOINT}

2024-10-30 14:52:04,204   INFO  **********************Start logging**********************
2024-10-30 14:52:04,648   INFO  data_path        /content/drive/MyDrive/ColabNotebooks/IECO/point2roof/dataset_colob
2024-10-30 14:52:04,648   INFO  cfg_file         ./model_cfg.yaml
2024-10-30 14:52:04,649   INFO  batch_size       1
2024-10-30 14:52:04,649   INFO  gpu              0
2024-10-30 14:52:04,650   INFO  test_tag         test_run_colab
2024-10-30 14:52:04,650   INFO  
cfg.DATA = edict()
2024-10-30 14:52:04,650   INFO  cfg.DATA.NPOINT: 1024
2024-10-30 14:52:04,650   INFO  
cfg.MODEL = edict()
2024-10-30 14:52:04,650   INFO  
cfg.MODEL.PointNet2 = edict()
2024-10-30 14:52:04,651   INFO  cfg.MODEL.PointNet2.PosRadius: 0.15
2024-10-30 14:52:04,651   INFO  
cfg.MODEL.PointNet2.LossWeight = edict()
2024-10-30 14:52:04,651   INFO  cfg.MODEL.PointNet2.LossWeight.cls_weight: 1.0
2024-10-30 14:52:04,651   INFO  cfg.MODEL.PointNet2.LossWeight.reg_weight: 1.0
2024-10-30 14:52:04,651   INFO  
cfg.MO